In [ ]:
import boto3
import emr_configuration_and_steps
import logging
import traceback

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s')

fh=logging.FileHandler("emr.log")

fh.setFormatter(formatter)

logger.addHandler(fh)


def lambda_handler(event,context):
    logger.info("starting the lambda function for spawning EMR")
    
    try:
        emr_provisioning = boto3.client('emr',region_name='us-east-1',aws_access_key_id=os.environ['aws_access_keys'],\
                                aws_secret_access_key=os.environ['aws_secret_key'])
        cluster_id = emr_provisioning.run_job_flow(
            Name='spark_job_cluster',
            VisibleToAllUsers=emr_configuration_and_steps.visible_to_all_users,
            LogUri=emr_configuration_and_steps.log_uri,
            ReleaseLabel=emr_configuration_and_steps.release_label,
            Applications=emr_configuration_and_steps.applications,
            Tags=emr_configuration_and_steps.tags,
            Instances=emr_configuration_and_steps.instances,
            Steps=emr_configuration_and_steps.steps,
            Configurations=emr_configuration_and_steps.configurations,
            ScaleDownBehavior=emr_configuration_and_steps.scale_down_behavior,
            ServiceRole=emr_configuration_and_steps.service_role,
            JobFlowRole=emr_configuration_and_steps.job_flow_role
        )
        logger.info("EMR is created successfully")
        return cluster_id['JobFlowId']
    except Exception as e:
        traceback.print_exc()
        raise Exception(e)